<a href="https://colab.research.google.com/github/ykitaguchi77/CorneAI/blob/main/Iris_color_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
import pandas as pd

orig_csv_path = '/content/drive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/Iris_color/corneAI_iris_color.csv'
dst_csv_path = '/content/drive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/Iris_color/corneAI_iris_color_submit.csv'


# Load the CSV file
df = pd.read_csv(orig_csv_path)


# Adding a new column 'image_id' to the dataframe
df['image_id'] = df.groupby('classification').cumcount() + 1
df['image_id'] = df['classification'] + '_' + df['image_id'].astype(str)

reordered_columns = ['image_id', 'URL', 'classification', 'comment']
df = df[reordered_columns]

# Save the dataframe to the specified path
df.to_csv(dst_csv_path, index=False)

# Display the modified dataframe
df


,image_id,URL,classification,comment
0,APAC_1,https://img.grepmed.com/uploads/54/angleclosur...,APAC,NaN
1,APAC_2,https://kellogg.umich.edu/theeyeshaveit/tehi_i...,APAC,NaN
2,APAC_3,https://www.researchgate.net/publication/28304...,APAC,NaN
3,APAC_4,https://www.tekportal.net/wp-content/uploads/2...,APAC,NaN
4,APAC_5,https://journals.sagepub.com/cms/10.4137/OED.S...,APAC,NaN
...,...,...,...,...
232,tumor_22,https://www.oftalvist.es/public/especialidades...,tumor,pterygium
233,tumor_23,https://t1.uc.ltmcdn.com/es/posts/9/2/3/como_e...,tumor,pterygium
234,tumor_24,https://cdn0.remedios-naturales.org/es/posts/2...,tumor,pinguécula
235,tumor_25,https://www.imoc.com.ar/wp/wp-content/uploads/...,tumor,pterygium
